## Genera Sequenze di Kraft ##

In [9]:
    
from itertools import combinations_with_replacement
import pyomo.environ as pyo
import numpy as np
from scipy.optimize import linear_sum_assignment
import json
import ast

class SequenceManipulator:
    """
    A class to perform various sequence manipulation operations.
    """

    debug_ = False

    def __init__(self):
        """
        Initializes the SequenceManipulator with an empty dictionary.
        """
        self.sequenze = {'2_2': {'sequenza': [2, 2], 'num_taxa': 3, 'entropia': 2, 'distribuzione': {2:2}}}
        self.meta_sequenze = {}

    def pprint(self):
        """
        Prints the elents of the dictionary
        """
        for k, v in self.meta_sequenze.items(): print(k,v)
        print("------------------------")
        for k, v in self.sequenze.items(): print(k,v)

    def generate_lists_with_replacements(self, numbers: list) -> dict:
        """
        Generates a dictionary of new lists for each element in the input list.
        Each element in the list is substituted with two elements that are both equal to the element plus 1.
        
        Parameters:
        numbers (list): The input list of numbers.
        
        Returns:
        dict: A dictionary where keys are strings obtained by joining the original list elements separated by underscores,
              and values are the new lists.
        """
        def count_numbers(_numbers):
                """
                Counts how many times each number appears in the given list.

                Parameters:
                numbers (list): The list of numbers to count.

                Returns:
                dict: A dictionary where keys are the numbers and values are the counts.
                """
                counts = {}
                for _number in _numbers:
                    if _number in counts:
                        counts[_number] += 1
                    else:
                        counts[_number] = 1
                return counts
        new_sequenze = {}
        numbers_len = len(numbers)
        for k in range(numbers_len):
            # Create a copy of the original list
            new_list = numbers[:]
            # Replace the element at index k with two elements both equal to numbers[k] + 1
            new_list[k:k+1] = [numbers[k] + 1, numbers[k] + 1]
            # Abort the new list if its kth element is greater than its k+1th
            if (k+1 < numbers_len) and new_list[k+1] > new_list[k+2]: continue
            # Create the key by joining the elements of the original list with underscores
            key = '_'.join(map(str, new_list))
            # Add the new list to the result dictionary with the generated key
            new_sequenze[key] = {"sequenza": new_list,
                                  "num_taxa": len(new_list) + 1,
                                  "entropia":  int(sum(i * 2 ** (numbers_len - i) for i in new_list)),
                                  'distribuzione': count_numbers(new_list)}
        return new_sequenze

    def merge_with_generated_lists(self) -> dict:
        """
        Merges the class dictionary of lists with the dictionary generated by generate_lists_with_replacements.
        
        Parameters:
        
        Returns:
        dict: A merged dictionary containing the original input dictionary and the generated lists dictionary.
        """
        # Initialize the merged dictionary with the input dictionary
        merged_dict = self.sequenze.copy()

        # Iterate through each list in the input dictionary
        for key, number_list in self.sequenze.items():
            # Generate lists with replacements for the current list
            generated_dict = self.generate_lists_with_replacements(number_list["sequenza"])
            # Merge the generated dictionary into the merged dictionary
            merged_dict.update(generated_dict)
        self.sequenze = merged_dict
        return merged_dict

    def generate_sequences(self, n: int) -> dict:
        """
        Calls the merge_with_generated_lists method n times and counts the number of lists for each possible length.
        
        Parameters:
        input_dict (dict): A dictionary where keys are strings and values are lists of numbers.
        n (int): The number of times to call merge_with_generated_lists.
        
        Returns:
        dict: A dictionary where keys are list lengths and values are the count of lists with that length.
        """
        merged_dict = self.sequenze

        # Call merge_with_generated_lists n times
        for _ in range(n):
            merged_dict = self.merge_with_generated_lists()

        # Initialize a dictionary to count the lengths of the lists
        self.meta_sequenze = {}

        # Iterate through the lists in the merged dictionary and count their lengths
        for lst in merged_dict.values():
            num_taxa = lst["num_taxa"]
            if num_taxa in self.meta_sequenze:
                self.meta_sequenze[num_taxa]["num_sequenze"] += 1
            else:
                self.meta_sequenze[num_taxa] = {"num_sequenze": 1,
                                              "num_archi": 2*num_taxa-3,
                                              "entropia":(2*num_taxa-3)*2**(num_taxa-2)}



        self.sequenze = merged_dict
        return self.meta_sequenze

    def find_combinations_with_entropy(self, num_taxa: int, leggi_combinazioni = None, feasible_res = True) -> list:
        """
        Selects the items of the dictionary whose 'num_taxa' is equal to the input integer and
        returns a list of all the combinations of the selected items whose sum of their 'entropia' equals
        the target entropy. An item can be selected multiple times.

        Parameters:
        num_taxa (int): The 'num_taxa' value to filter the dictionary items.

        Returns:
        list: A list of all combinations whose sum of 'entropia' equals the target entropy.
              Each combination is represented as a list of tuples, where each tuple contains
              the dictionary key and the number of times the item is included in the combination.
        """
        target_entropy = self.meta_sequenze[num_taxa]["entropia"]

        # Filter the items based on 'num_taxa'
        filtered_items = {k: v for k, v in self.sequenze.items() if v['num_taxa'] == num_taxa}
        
        # Prepare a list to store the results
        result = []

        # Extract the keys and their respective entropies
        keys = list(filtered_items.keys())
        entropies = [filtered_items[key]['entropia'] for key in keys]
        min_entropia = min(filtered_items[key]['entropia'] for key in keys)

        # Helper function to find combinations
        def find_combinations(current_combo, current_sum, index):
            if current_sum == target_entropy:
                if len(current_combo) != num_taxa: return
                # Convert current_combo to the required format
                combo_dict = {}
                for key in current_combo:
                    if key in combo_dict:
                        combo_dict[key] += 1
                    else:
                        combo_dict[key] = 1
                result.append(list(combo_dict.items()))
                if self.debug_: print(list(combo_dict.items()))
                return
            if current_sum > target_entropy - min_entropia or index >= len(keys) or len(current_combo) >= num_taxa:
                return
            # Include the current item
            #print(keys[index], keys[index][0], current_combo)
            if keys[index][0] == "2": 
                if len(current_combo) >= num_taxa - 1: return
                find_combinations(current_combo + [keys[index], keys[index]], current_sum + 2*entropies[index], index)
            else: find_combinations(current_combo + [keys[index]], current_sum + entropies[index], index)
            # Exclude the current item and move to the next
            find_combinations(current_combo, current_sum, index + 1)

        # legge le combinazioni da file
        def read_file_to_list(filename):
            with open(filename, 'r') as file:
                lines = file.readlines()
                result = []
                for line in lines:
                    line = line.strip()
                    if line:
                        try:
                            # Convert the string representation of the list back to an actual list
                            list_in_line = ast.literal_eval(line)
                            result.append(list_in_line)
                        except ValueError as e:
                            print(f"Error processing line: {line}\nError: {e}")
                return result
            
        # Start finding combinations
        if leggi_combinazioni is None: find_combinations([], 0, 0)
        else: result = read_file_to_list(leggi_combinazioni)
        
        self.meta_sequenze[num_taxa]["combinazioni_KM"] = len(result)
        # Remove the combinations which assumes an odd number of istances in presence of a 2
        total_result = [
                        el for el in result
                        if (not any(c[0][0] == "2" and c[1] % 2 != 0 for c in el)) and sum(c[1] for c in el) == num_taxa
                    ]
        self.meta_sequenze[num_taxa]["combinazioni_KM2"] = len(total_result)

        if leggi_combinazioni is None:
            print("--------------------------")
            print("Combinazioni totali")
            if not self.debug_:
                for c in total_result: print(c)
            print("--------------------------")

        if self.debug_: return [], total_result

        # Indicates the combinations which violate symmetry constraints
        def check_combination(combo):
            #print(combo)
            for el in combo:
                threshold = 2 if el[0][0] == "2" else 1
                #print(filtered_items[el[0]])
                for kk, vv in filtered_items[el[0]]['distribuzione'].items():
                    if kk == 2: continue
                    potential_rows = sum((ee[1] - (threshold if ee[0] == el[0] else 0))*(1 if filtered_items[ee[0]]['distribuzione'].get(kk,0) >= threshold else 0) for ee in combo)
                    #print(kk, vv, potential_rows)
                    if potential_rows < vv: return False

                # violazione di assegnamento tra elemnti di una riga e i tipi delle altre righe a disposizione    
                cost_ = []   
                infinito = 10000000
                for kk, vv in filtered_items[el[0]]['distribuzione'].items():  
                    if kk == 2: continue
                    cost_vv = []
                    for ee in combo: 
                        for i in range(ee[1] - (threshold if ee[0] == el[0] else 0)):   
                            cost_vv.append(1 if filtered_items[ee[0]]['distribuzione'].get(kk,0) >= threshold else infinito)
                    for ii in range(vv): cost_.append(cost_vv)
                cost = np.array(cost_) 
                row_ind, col_ind = linear_sum_assignment(cost)
                if cost[row_ind, col_ind].sum() > infinito: return False

                #violazione condizioni triangolari forti
                #la somma dei due valori piu' alti di ogni tipo deve eccedere di almeno 2 il valore massimo di ogli altro tipo.
                val_max = filtered_items[el[0]]['sequenza'][-1]
                for i in range(2, num_taxa): 
                    secondo_valore = filtered_items[el[0]]['sequenza'][-i]
                    soglia_trian_ineq = 2#2*threshold
                    if any((el[0] != ee[0]) and (filtered_items[ee[0]]['sequenza'][-(i-1)] + soglia_trian_ineq > val_max + secondo_valore)  for ee in combo): return False             
            return True
        
        if feasible_res: feasible_result = [el for el in total_result if check_combination(el)] 
        else: feasible_result = []

        # Indicates the combinations which violate assignment constraints
        def check_combination2(combo):
            #combo = list(reversed(combo))
            combo_seq = [i for i in range(len(combo)) for k in range(combo[i][1])]  
            print(combo,combo_seq)
            for el in combo:
                threshold = 2 if el[0][0] == "2" else 1
                # violazione di assegnamento tra elemnti di una riga e i tipi delle altre righe a disposizione    
                cost_ = []  
                infinito = 10000000
                for kk, vv in filtered_items[el[0]]['distribuzione'].items():  
                    if kk == 2: continue
                    cost_vv = []
                    for ee in combo: 
                        for i in range(ee[1] - (threshold if ee[0] == el[0] else 0)):   
                            cost_vv.append(kk if filtered_items[ee[0]]['distribuzione'].get(kk,0) >= threshold else infinito)
                    for ii in range(vv): cost_.append(cost_vv)
                cost = np.array(cost_) 
                print(cost)

                # trova tutte le possibili assegnazioni
                def find_assignment_solutions_within_threshold(cost_matrix, cost_threshold):
                    row_ind, col_ind = linear_sum_assignment(cost_matrix)
                    min_cost = cost_matrix[row_ind, col_ind].sum()
                    if min_cost >= cost_threshold: return []
                    solutions = []#{}
                    num_rows, num_cols = cost_matrix.shape

                    def backtrack(curr_cost, assigned_rows, assigned_cols):
                        if len(assigned_rows) == num_rows:
                            if curr_cost < cost_threshold:
                                #print(curr_cost, sum(cost[row][col] for row, col in zip(assigned_rows, assigned_cols)))
                                #solutions[curr_cost] = [(row, col, cost[row][col]) for row, col in zip(assigned_rows, assigned_cols)]
                                solution = [(row, col, combo_seq[col+threshold]) for row, col in zip(assigned_rows, assigned_cols)]
                                if any(all(solution[i][2] == s[i][2] for i in range(num_rows)) for s in solutions): pass
                                else: solutions.append(solution)
                            return
                        row = len(assigned_rows)
                        for col in range(num_cols):
                            if col not in assigned_cols:
                                next_cost = curr_cost + cost_matrix[row, col]
                                if next_cost < cost_threshold:
                                    backtrack(next_cost, assigned_rows + [row], assigned_cols + [col])
                    backtrack(0, [], [])
                    return solutions

                assegnazioni_alternative = find_assignment_solutions_within_threshold(cost, infinito)
                #print(el[0], " --> ", combo, " --> ",assegnazioni_alternative)
                print(combo, " --> ",assegnazioni_alternative)
                break

            return True
        
        def determine_tau(combo, triang = False):
            """
            Le sequenze che iniziano per 2 devono essere le prime
            """
            #combo = list(reversed(combo))
            a = [[0] + filtered_items[combo[-i][0]]['sequenza'] for i in range(1,len(combo)+1) for _ in range(combo[-i][1])]

            # Get the size of the matrix
            n = len(a)
            cbc_path = "C:/Program Files/Cbc/bin/cbc.exe"

            # Create a model
            model = pyo.ConcreteModel(str(combo))

            # Sets
            model.I = pyo.RangeSet(0, n-1)
            model.J = pyo.RangeSet(0, n-1)
            model.K = pyo.RangeSet(0, n-1)

            # Variables
            model.x = pyo.Var(model.I, model.K, model.J, domain=pyo.Binary)
            model.t = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

            # Objective
            model.obj = pyo.Objective(expr=model.x[0, n-1, 0], sense=pyo.maximize)

            # Constraints
            # valori delle righe di tau
            def t_i_j_rule(model, i, j):
                return model.t[i, j] == sum(a[i][k] * model.x[i, k, j] for k in model.K)
            model.t_i_j = pyo.Constraint(model.I, model.J, rule=t_i_j_rule)
            # valori delle colonne di tau rispettando la simmetria
            def t_j_i_rule(model, i, j):
                return model.t[i, j] == sum(a[j][k] * model.x[j, k, i] for k in model.K)
            model.t_j_i = pyo.Constraint(model.I, model.J, rule=t_j_i_rule)
            # tutti gli elementi della riga i devono essere assegnati
            def sum_x_rule(model, j, k):
                return sum(model.x[j, k, i] for i in model.I) == 1
            model.sum_x = pyo.Constraint(model.J, model.K, rule=sum_x_rule)
            # tutti gli elementi della riga i devono essere assegnati
            def sum_xk_rule(model, j, i):
                return sum(model.x[j, k, i] for k in model.K) == 1
            model.sum_xk = pyo.Constraint(model.J, model.I, rule=sum_xk_rule)
            if triang:
                # disuguaglianze triangolari forti
                def t_0_j_t_j_k_rule(model, j, k):
                    if j > 0 and k > 0 and k != j: return model.t[0, j] + model.t[j, k] >= 2 + model.t[0, k]
                    else: return pyo.Constraint.Skip
                model.t_0_j_t_j_k = pyo.Constraint(model.J, model.K, rule=t_0_j_t_j_k_rule)
                # disuguaglianze triangolari forti
                def t_j_0_t_0_k_rule(model, j, k):
                    if j > 0 and k > 0 and k != j: return model.t[0, j] + model.t[0, k] >= 2 + model.t[j, k]
                    else: return pyo.Constraint.Skip
                model.t_j_0_t_0_k = pyo.Constraint(model.J, model.K, rule=t_j_0_t_0_k_rule)
            # 0 sulle diagonali
            def x_i_0_i_rule(model, i):
                return model.x[i, 0, i] == 1
            model.x_i_0_i = pyo.Constraint(model.I, rule=x_i_0_i_rule)
            # 2 sulle diagonali minori quando righe doppie
            def x_i_1_i_plus_minus_1_rule(model, i):
                if (a[i][1] == 2): return model.x[i, 1, i + 1] == 1 if (i % 2 == 0) else model.x[i, 1, i - 1] == 1
                else: return pyo.Constraint.Skip
            model.x_i_1_i_plus_minus_1 = pyo.Constraint(model.I, rule=x_i_1_i_plus_minus_1_rule)
            # gestione righe doppie
            def x_i_k_j_rule(model, i, k, j):
                if (a[i][1] == 2) and (i % 2 == 0) and (i < n-1) and (j != i) and (j != i + 1):
                    return model.x[i, k, j] == model.x[i+1, k, j]
                else: return pyo.Constraint.Skip
            model.x_i_k_j = pyo.Constraint(model.I, model.K, model.J, rule=x_i_k_j_rule)
            # gestione valori doppi sulle righe
            def x_i_k_plus_1_0_rule(model, i, k):
                if (k < n-1) and (a[i][k] == a[i][k + 1]):
                    return model.x[i, k + 1, 0] == 0
                else: return pyo.Constraint.Skip
            model.x_i_k_plus_1_0 = pyo.Constraint(model.I, model.K, rule=x_i_k_plus_1_0_rule)
            # gestione valori doppi sulle righe
            def x_i_k_plus_1_j_plus_1_rule(model, i, k, j):
                if (k < n-1) and (a[i][k] == a[i][k + 1]) and (j >= 1):
                    return model.x[i, k + 1, j] <= sum(model.x[i, k, r] for r in range(0,j))
                else: return pyo.Constraint.Skip
            model.x_i_k_plus_1_j_plus_1 = pyo.Constraint(model.I, model.K, model.J, rule=x_i_k_plus_1_j_plus_1_rule)

            # Solve the model using a solver
            #solver = pyo.SolverFactory('xpress')
            solver = pyo.SolverFactory('cbc', executable=cbc_path)

            # Solve the problem
            results = solver.solve(model, tee=False, timelimit=3600, options={"mipgap": 0.05})

            if (results.solver.termination_condition == pyo.TerminationCondition.infeasible): return(None)
            
            # Extract results
            #t_values = {(i, j): pyo.value(model.t[i, j]) for i in model.I for j in model.J}
            t_values = [[pyo.value(model.t[i, j]) for i in model.I] for j in model.J]

            return t_values 
        
        #for el in feasible_result: check_combination2(el)
        self.meta_sequenze[num_taxa]["combinazioni_KM2Symm"] = len(feasible_result)


        
        no_triang_feasible_result = []
        for el in feasible_result: 
            t_values = determine_tau(el)
            #print(el)
            if t_values:
                #for e in t_values: print(e)
                no_triang_feasible_result.append(el)
            #else: print("infeasible")
        self.meta_sequenze[num_taxa]["combinazioni_KM2SymmNoTriang"] = len(no_triang_feasible_result)

        if not feasible_res: no_triang_feasible_result = total_result
        
        triang_feasible_result = []
        with open("output.txt", "a") as f:
            for el in no_triang_feasible_result: 
                t_values = determine_tau(el,triang = True)
                print(el)
                if t_values:
                    for e in t_values: 
                        for ee in e:
                            print(ee, end = " ")
                        print("")
                    triang_feasible_result.append(el)
                else: 
                    print(el, file=f)
                    print("infeasible")
        self.meta_sequenze[num_taxa]["combinazioni_KM2SymmTriang"] = len(triang_feasible_result)

        return triang_feasible_result, total_result

    def solve_ilp(self, a: list, b: int, n: int):
        """
        Solves the integer linear programming problem:
        max sum(a_i * x_i)
        subject to:
        sum(a_i * x_i) <= b
        sum(x_i) = n
        x_i: integer
        
        Parameters:
        a (list): Coefficients for the objective function.
        b (int): Upper bound for the constraint sum(a_i * x_i) <= b.
        n (int): The exact sum of the decision variables x_i.
        
        Returns:
        dict: The optimal values of the decision variables x_i.
        """
        cbc_path = "C:/Program Files/Cbc/bin/cbc.exe"
        #cbc_path = "C:/Program Files/Cbc/bin/cbc.exe"

        # Number of variables
        num_vars = len(a)
        
        # Create a Pyomo model
        model = pyo.ConcreteModel()

        # Define the decision variables x_i (integer, non-negative)
        model.x = pyo.Var(range(num_vars), within=pyo.NonNegativeIntegers)

        # Objective function: max sum(a_i * x_i)
        model.objective = pyo.Objective(expr=sum(a[i] * model.x[i] for i in range(num_vars)), sense=pyo.maximize)

        # Constraint: sum(a_i * x_i) <= b
        model.constraint1 = pyo.Constraint(expr=sum(a[i] * model.x[i] for i in range(num_vars)) <= b)

        # Constraint: sum(x_i) = n
        model.constraint2 = pyo.Constraint(expr=sum(model.x[i] for i in range(num_vars)) == n)
        #model.pprint()

        # Solve the model using a solver
        solver = pyo.SolverFactory('xpress')
        #solver = pyo.SolverFactory('cbc', executable=cbc_path)
        result = solver.solve(model)

        # Extract the optimal values of the decision variables
        solution = {i: model.x[i].value for i in range(num_vars)}
        
        return solution
    

def main(n, combinazioni = False, leggi_combinazioni = None, feasible_res = True, cerca_due = False, stampa_tabella = False):
    # Instantiate the StringManipulator class with a sample string
    manipulator = SequenceManipulator()
    manipulator.generate_sequences(n-3)
    filtered_items = {k: v for k, v in manipulator.sequenze.items() if v['num_taxa'] == n}
    entropia = manipulator.meta_sequenze[n]["entropia"]

    
    print("n = ",n)
    print("Entropia = ",entropia)
    print("------------------------")
    for k, v in filtered_items.items(): print(k,v)
    print("------------------------")
    # for k, v in filtered_items.items(): print(v['sequenza'], ',')
    print("------------------------")

    if combinazioni:  
        combinazioni_ammissibili, combinazioni_totali = manipulator.find_combinations_with_entropy(num_taxa = n, 
                                                                                                   leggi_combinazioni = leggi_combinazioni,
                                                                                                   feasible_res = feasible_res)
        print(json.dumps(manipulator.meta_sequenze[n], indent=4))
        print("Num_combinazioni_ammissibili = ",len(combinazioni_ammissibili))
        print("Num_combinazioni_totali = ",len(combinazioni_totali))
        for c in combinazioni_ammissibili: print(c)
        if stampa_tabella:
            print("------------------------")
            output = "|".join("c" for i in range(len(combinazioni_totali)+1))
            print(f"\\begin{{tabular}}{{|{output}|}}\\\\ \\hline")
            output = " &".join(f"{i+1}" for i in range(len(combinazioni_totali)))
            print(f"&{output}\\\\ \\hline")
            for k, v in filtered_items.items(): 
                print(v['sequenza'], end="")
                for el in combinazioni_totali:
                    coeff = next((c[1] for c in el if c[0] == k), None)
                    if coeff: print(f"&{coeff}",end="")
                    else: print("& ",end="")
                print("\\\\ \hline")
            print(f"\\end{{tabular}}")


    if cerca_due:
        a = [v["entropia"] for k,v in filtered_items.items() if k[0] != "2"]
        optimal_values = manipulator.solve_ilp(a, entropia, n)
        for k, v in optimal_values.items(): 
            if v > 0: print(k,v,a[k])

    #manipulator.pprint()
    #print(manipulator.sequenze)

if __name__ == "__main__":
    n = 9
    file_combinazioni = f"./Data/TutteCombinazioni{n}.txt"
    # file_combinazioni = f"./Data/TutteCombinazioni{n}Triang.txt"#
    # file_combinazioni = f"./Data/TutteCombinazioni{n}Triang_ToCheck.txt"#
    #file_combinazioni = f"./Data/Test{n}.txt"#
    main(n,combinazioni = True, leggi_combinazioni = file_combinazioni, feasible_res = True, cerca_due = False, stampa_tabella = False)


n =  9
Entropia =  1920
------------------------
4_4_4_4_4_4_4_4 {'sequenza': [4, 4, 4, 4, 4, 4, 4, 4], 'num_taxa': 9, 'entropia': 256, 'distribuzione': {4: 8}}
3_4_4_4_4_4_5_5 {'sequenza': [3, 4, 4, 4, 4, 4, 5, 5], 'num_taxa': 9, 'entropia': 248, 'distribuzione': {3: 1, 4: 5, 5: 2}}
3_3_4_4_5_5_5_5 {'sequenza': [3, 3, 4, 4, 5, 5, 5, 5], 'num_taxa': 9, 'entropia': 240, 'distribuzione': {3: 2, 4: 2, 5: 4}}
3_3_4_4_4_5_6_6 {'sequenza': [3, 3, 4, 4, 4, 5, 6, 6], 'num_taxa': 9, 'entropia': 236, 'distribuzione': {3: 2, 4: 3, 5: 1, 6: 2}}
3_3_3_5_5_5_6_6 {'sequenza': [3, 3, 3, 5, 5, 5, 6, 6], 'num_taxa': 9, 'entropia': 228, 'distribuzione': {3: 3, 5: 3, 6: 2}}
3_3_3_4_6_6_6_6 {'sequenza': [3, 3, 3, 4, 6, 6, 6, 6], 'num_taxa': 9, 'entropia': 224, 'distribuzione': {3: 3, 4: 1, 6: 4}}
3_3_3_4_5_6_7_7 {'sequenza': [3, 3, 3, 4, 5, 6, 7, 7], 'num_taxa': 9, 'entropia': 222, 'distribuzione': {3: 3, 4: 1, 5: 1, 6: 1, 7: 2}}
2_4_5_5_5_5_5_5 {'sequenza': [2, 4, 5, 5, 5, 5, 5, 5], 'num_taxa': 9, 'entrop

ln =  8
Entropia =  832
------------------------
3_4_4_4_4_4_4 {'sequenza': [3, 4, 4, 4, 4, 4, 4], 'num_taxa': 8, 'entropia': 120, 'distribuzione': {3: 1, 4: 6}}
3_3_4_4_4_5_5 {'sequenza': [3, 3, 4, 4, 4, 5, 5], 'num_taxa': 8, 'entropia': 116, 'distribuzione': {3: 2, 4: 3, 5: 2}}
3_3_3_5_5_5_5 {'sequenza': [3, 3, 3, 5, 5, 5, 5], 'num_taxa': 8, 'entropia': 112, 'distribuzione': {3: 3, 5: 4}}
3_3_3_4_5_6_6 {'sequenza': [3, 3, 3, 4, 5, 6, 6], 'num_taxa': 8, 'entropia': 110, 'distribuzione': {3: 3, 4: 1, 5: 1, 6: 2}}
2_4_4_5_5_5_5 {'sequenza': [2, 4, 4, 5, 5, 5, 5], 'num_taxa': 8, 'entropia': 104, 'distribuzione': {2: 1, 4: 2, 5: 4}}
2_4_4_4_5_6_6 {'sequenza': [2, 4, 4, 4, 5, 6, 6], 'num_taxa': 8, 'entropia': 102, 'distribuzione': {2: 1, 4: 3, 5: 1, 6: 2}}
2_3_5_5_5_6_6 {'sequenza': [2, 3, 5, 5, 5, 6, 6], 'num_taxa': 8, 'entropia': 98, 'distribuzione': {2: 1, 3: 1, 5: 3, 6: 2}}
2_3_4_6_6_6_6 {'sequenza': [2, 3, 4, 6, 6, 6, 6], 'num_taxa': 8, 'entropia': 96, 'distribuzione': {2: 1, 3: 1, 4: 1, 6: 4}}
2_3_4_5_6_7_7 {'sequenza': [2, 3, 4, 5, 6, 7, 7], 'num_taxa': 8, 'entropia': 95, 'distribuzione': {2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 2}}
------------------------
{'num_sequenze': 9, 'num_archi': 13, 'entropia': 832, 'combinazioni_KM': 185, 'combinazioni_KM2': 28, 'combinazioni_KM2Symm': 6}
Num_combinazioni_ammissibili =  6
Num_combinazioni_totali =  28
- 5:  [('3_4_4_4_4_4_4', 1), ('3_3_3_5_5_5_5', 1), ('3_3_3_4_5_6_6', 2), ('2_3_4_5_6_7_7', 4)]
7:    [('3_4_4_4_4_4_4', 1), ('3_3_3_5_5_5_5', 1), ('2_4_4_4_5_6_6', 4), ('2_3_4_6_6_6_6', 2)]
10:   [('3_3_4_4_4_5_5', 2), ('3_3_3_4_5_6_6', 2), ('2_3_4_5_6_7_7', 4)]
11:   [('3_3_4_4_4_5_5', 2), ('2_4_4_5_5_5_5', 2), ('2_3_5_5_5_6_6', 4)]
- 12: [('3_3_4_4_4_5_5', 2), ('2_4_4_4_5_6_6', 4), ('2_3_4_6_6_6_6', 2)]
28:   [('2_4_4_5_5_5_5', 8)]

n =  9
Entropia =  1920
------------------------
4_4_4_4_4_4_4_4 {'sequenza': [4, 4, 4, 4, 4, 4, 4, 4], 'num_taxa': 9, 'entropia': 256, 'distribuzione': {4: 8}}
3_4_4_4_4_4_5_5 {'sequenza': [3, 4, 4, 4, 4, 4, 5, 5], 'num_taxa': 9, 'entropia': 248, 'distribuzione': {3: 1, 4: 5, 5: 2}}
3_3_4_4_5_5_5_5 {'sequenza': [3, 3, 4, 4, 5, 5, 5, 5], 'num_taxa': 9, 'entropia': 240, 'distribuzione': {3: 2, 4: 2, 5: 4}}
3_3_4_4_4_5_6_6 {'sequenza': [3, 3, 4, 4, 4, 5, 6, 6], 'num_taxa': 9, 'entropia': 236, 'distribuzione': {3: 2, 4: 3, 5: 1, 6: 2}}
3_3_3_5_5_5_6_6 {'sequenza': [3, 3, 3, 5, 5, 5, 6, 6], 'num_taxa': 9, 'entropia': 228, 'distribuzione': {3: 3, 5: 3, 6: 2}}
3_3_3_4_6_6_6_6 {'sequenza': [3, 3, 3, 4, 6, 6, 6, 6], 'num_taxa': 9, 'entropia': 224, 'distribuzione': {3: 3, 4: 1, 6: 4}}
3_3_3_4_5_6_7_7 {'sequenza': [3, 3, 3, 4, 5, 6, 7, 7], 'num_taxa': 9, 'entropia': 222, 'distribuzione': {3: 3, 4: 1, 5: 1, 6: 1, 7: 2}}
2_4_5_5_5_5_5_5 {'sequenza': [2, 4, 5, 5, 5, 5, 5, 5], 'num_taxa': 9, 'entropia': 216, 'distribuzione': {2: 1, 4: 1, 5: 6}}
2_4_4_5_5_5_6_6 {'sequenza': [2, 4, 4, 5, 5, 5, 6, 6], 'num_taxa': 9, 'entropia': 212, 'distribuzione': {2: 1, 4: 2, 5: 3, 6: 2}}
2_4_4_4_6_6_6_6 {'sequenza': [2, 4, 4, 4, 6, 6, 6, 6], 'num_taxa': 9, 'entropia': 208, 'distribuzione': {2: 1, 4: 3, 6: 4}}
2_4_4_4_5_6_7_7 {'sequenza': [2, 4, 4, 4, 5, 6, 7, 7], 'num_taxa': 9, 'entropia': 206, 'distribuzione': {2: 1, 4: 3, 5: 1, 6: 1, 7: 2}}
2_3_5_5_6_6_6_6 {'sequenza': [2, 3, 5, 5, 6, 6, 6, 6], 'num_taxa': 9, 'entropia': 200, 'distribuzione': {2: 1, 3: 1, 5: 2, 6: 4}}
2_3_5_5_5_6_7_7 {'sequenza': [2, 3, 5, 5, 5, 6, 7, 7], 'num_taxa': 9, 'entropia': 198, 'distribuzione': {2: 1, 3: 1, 5: 3, 6: 1, 7: 2}}
2_3_4_6_6_6_7_7 {'sequenza': [2, 3, 4, 6, 6, 6, 7, 7], 'num_taxa': 9, 'entropia': 194, 'distribuzione': {2: 1, 3: 1, 4: 1, 6: 3, 7: 2}}
2_3_4_5_7_7_7_7 {'sequenza': [2, 3, 4, 5, 7, 7, 7, 7], 'num_taxa': 9, 'entropia': 192, 'distribuzione': {2: 1, 3: 1, 4: 1, 5: 1, 7: 4}}
2_3_4_5_6_7_8_8 {'sequenza': [2, 3, 4, 5, 6, 7, 8, 8], 'num_taxa': 9, 'entropia': 191, 'distribuzione': {2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 2}}
------------------------
{'num_sequenze': 16, 'num_archi': 15, 'entropia': 1920, 'combinazioni_KM': 9570, 'combinazioni_KM2': 469, 'combinazioni_KM2Symm': 25}
Num_combinazioni_ammissibili =  22
Num_combinazioni_totali =  469
OK - 4cherry  [('4_4_4_4_4_4_4_4', 1), ('2_4_4_4_6_6_6_6', 8)]
OK - 3cherry  [('3_4_4_4_4_4_5_5', 1), ('3_3_4_4_5_5_5_5', 1), ('3_3_3_4_6_6_6_6', 1), ('2_4_4_4_5_6_7_7', 4), ('2_3_4_5_7_7_7_7', 2)]
OK - 3cherry  [('3_4_4_4_4_4_5_5', 1), ('3_3_4_4_4_5_6_6', 1), ('3_3_3_5_5_5_6_6', 1), ('2_4_4_5_5_5_6_6', 2), ('2_3_5_5_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 2)]
[('3_4_4_4_4_4_5_5', 1), ('3_3_4_4_4_5_6_6', 1), ('3_3_3_5_5_5_6_6', 1), ('2_4_4_4_5_6_7_7', 4), ('2_3_4_5_7_7_7_7', 2)]
[('3_4_4_4_4_4_5_5', 1), ('3_3_3_5_5_5_6_6', 2), ('2_4_4_4_6_6_6_6', 2), ('2_4_4_4_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 2)]
[('3_4_4_4_4_4_5_5', 1), ('3_3_3_5_5_5_6_6', 1), ('3_3_3_4_6_6_6_6', 1), ('3_3_3_4_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 4)]
OK - 3cherry [('3_3_4_4_5_5_5_5', 3), ('2_3_5_5_6_6_6_6', 6)]
[('3_3_4_4_5_5_5_5', 2), ('3_3_3_5_5_5_6_6', 1), ('3_3_3_4_5_6_7_7', 2), ('2_3_4_5_7_7_7_7', 4)]
OK - caterpillar  [('3_3_4_4_5_5_5_5', 1), ('3_3_4_4_4_5_6_6', 2), ('3_3_3_4_5_6_7_7', 2), ('2_3_4_5_6_7_8_8', 4)]
[('3_3_4_4_5_5_5_5', 1), ('3_3_4_4_4_5_6_6', 2), ('2_4_4_5_5_5_6_6', 2), ('2_3_5_5_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 2)]
[('3_3_4_4_5_5_5_5', 1), ('3_3_4_4_4_5_6_6', 2), ('2_4_4_4_5_6_7_7', 4), ('2_3_4_5_7_7_7_7', 2)]
[('3_3_4_4_5_5_5_5', 1), ('3_3_4_4_4_5_6_6', 1), ('3_3_3_5_5_5_6_6', 1), ('2_4_4_4_6_6_6_6', 2), ('2_4_4_4_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 2)]
[('3_3_4_4_5_5_5_5', 1), ('3_3_3_5_5_5_6_6', 2), ('2_4_5_5_5_5_5_5', 2), ('2_3_5_5_5_6_7_7', 4)]
[('3_3_4_4_5_5_5_5', 1), ('3_3_3_5_5_5_6_6', 2), ('2_4_4_5_5_5_6_6', 2), ('2_4_4_4_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 2)]
OK - 4cherry  [('3_3_4_4_5_5_5_5', 1), ('2_4_5_5_5_5_5_5', 2), ('2_4_4_5_5_5_6_6', 4), ('2_3_5_5_6_6_6_6', 2)]
[('3_3_4_4_4_5_6_6', 2), ('3_3_3_5_5_5_6_6', 1), ('3_3_3_4_5_6_7_7', 2), ('2_3_4_6_6_6_7_7', 4)]
[('3_3_4_4_4_5_6_6', 2), ('3_3_3_4_6_6_6_6', 3), ('2_3_4_6_6_6_7_7', 4)]
[('3_3_4_4_4_5_6_6', 2), ('3_3_3_4_6_6_6_6', 1), ('2_4_4_5_5_5_6_6', 2), ('2_3_5_5_6_6_6_6', 4)]
[('3_3_4_4_4_5_6_6', 1), ('3_3_3_5_5_5_6_6', 3), ('3_3_3_4_6_6_6_6', 1), ('2_3_4_6_6_6_7_7', 4)]
[('3_3_4_4_4_5_6_6', 1), ('3_3_3_5_5_5_6_6', 1), ('3_3_3_4_6_6_6_6', 1), ('2_4_4_5_5_5_6_6', 2), ('2_4_4_4_5_6_7_7', 2), ('2_3_5_5_5_6_7_7', 2)]
[('3_3_3_5_5_5_6_6', 3), ('3_3_3_4_5_6_7_7', 2), ('2_3_5_5_5_6_7_7', 4)]
[('3_3_3_4_6_6_6_6', 5), ('2_3_5_5_6_6_6_6', 4)]

In [ ]:
import pyomo.environ as pyo

def solve_milp(a):
    """
    Le sequenze che iniziano per 2 devono essere le prime
    """
    # Get the size of the matrix
    n = len(a)
    cbc_path = "C:/Program Files/Cbc/bin/cbc.exe"

    # Create a model
    model = pyo.ConcreteModel()

    # Sets
    model.I = pyo.RangeSet(0, n-1)
    model.J = pyo.RangeSet(0, n-1)
    model.K = pyo.RangeSet(0, n-1)

    # Variables
    model.x = pyo.Var(model.I, model.K, model.J, domain=pyo.Binary)
    model.t = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

    # Objective
    model.obj = pyo.Objective(expr=model.x[0, n-1, 0], sense=pyo.maximize)

    # Constraints
    # valori delle righe di tau
    def t_i_j_rule(model, i, j):
        return model.t[i, j] == sum(a[i][k] * model.x[i, k, j] for k in model.K)
    model.t_i_j = pyo.Constraint(model.I, model.J, rule=t_i_j_rule)
    # valori delle colonne di tau rispettando la simmetria
    def t_j_i_rule(model, i, j):
        return model.t[i, j] == sum(a[j][k] * model.x[j, k, i] for k in model.K)
    model.t_j_i = pyo.Constraint(model.I, model.J, rule=t_j_i_rule)
    # tutti gli elementi della riga i devono essere assegnati
    def sum_x_rule(model, j, k):
        return sum(model.x[j, k, i] for i in model.I) == 1
    model.sum_x = pyo.Constraint(model.J, model.K, rule=sum_x_rule)
    # tutti gli elementi della riga i devono essere assegnati
    def sum_xk_rule(model, j, i):
        return sum(model.x[j, k, i] for k in model.K) == 1
    model.sum_xk = pyo.Constraint(model.J, model.I, rule=sum_xk_rule)
    # # disuguaglianze triangolari forti
    # def t_0_j_t_j_k_rule(model, j, k):
    #     if j > 0 and k > 0 and k != j: return model.t[0, j] + model.t[j, k] >= 2 + model.t[0, k]
    #     else: return pyo.Constraint.Skip
    # model.t_0_j_t_j_k = pyo.Constraint(model.J, model.K, rule=t_0_j_t_j_k_rule)
    # # disuguaglianze triangolari forti
    # def t_j_0_t_0_k_rule(model, j, k):
    #     if j > 0 and k > 0 and k != j: return model.t[0, j] + model.t[0, k] >= 2 + model.t[j, k]
    #     else: return pyo.Constraint.Skip
    # model.t_j_0_t_0_k = pyo.Constraint(model.J, model.K, rule=t_j_0_t_0_k_rule)
    # 0 sulle diagonali
    def x_i_0_i_rule(model, i):
        return model.x[i, 0, i] == 1
    model.x_i_0_i = pyo.Constraint(model.I, rule=x_i_0_i_rule)
    # 2 sulle diagonali minori quando righe doppie
    def x_i_1_i_plus_minus_1_rule(model, i):
        if (a[i][1] == 2): return model.x[i, 1, i + 1] == 1 if (i % 2 == 0) else model.x[i, 1, i - 1] == 1
        else: return pyo.Constraint.Skip
    model.x_i_1_i_plus_minus_1 = pyo.Constraint(model.I, rule=x_i_1_i_plus_minus_1_rule)
    # gestione righe doppie
    def x_i_k_j_rule(model, i, k, j):
        if (a[i][1] == 2) and (i % 2 == 0) and (i < n-1) and (j != i) and (j != i + 1):
            return model.x[i, k, j] == model.x[i+1, k, j]
        else: return pyo.Constraint.Skip
    model.x_i_k_j = pyo.Constraint(model.I, model.K, model.J, rule=x_i_k_j_rule)
    # gestione valori doppi sulle righe
    def x_i_k_plus_1_0_rule(model, i, k):
        if (k < n-1) and (a[i][k] == a[i][k + 1]):
            return model.x[i, k + 1, 0] == 0
        else: return pyo.Constraint.Skip
    model.x_i_k_plus_1_0 = pyo.Constraint(model.I, model.K, rule=x_i_k_plus_1_0_rule)
    # gestione valori doppi sulle righe
    def x_i_k_plus_1_j_plus_1_rule(model, i, k, j):
        if (k < n-1) and (a[i][k] == a[i][k + 1]) and (j >= 1):
            return model.x[i, k + 1, j] <= sum(model.x[i, k, r] for r in range(0,j))
        else: return pyo.Constraint.Skip
    model.x_i_k_plus_1_j_plus_1 = pyo.Constraint(model.I, model.K, model.J, rule=x_i_k_plus_1_j_plus_1_rule)


    # Solve the model using a solver
    solver = pyo.SolverFactory('cbc', executable=cbc_path)

    # Solve the problem
    results = solver.solve(model, tee=False, timelimit=60, options={"mipgap": 0.05})

    if (results.solver.termination_condition == pyo.TerminationCondition.infeasible): return(None)
      
    # Extract results
    #t_values = {(i, j): pyo.value(model.t[i, j]) for i in model.I for j in model.J}
    t_values = [[pyo.value(model.t[i, j]) for i in model.I] for j in model.J]

    return t_values 

# Example usage
# combinazione 12 a = [[0,2,4,4,4,5,6,6], [0,2,4,4,4,5,6,6],[0,2,4,4,4,5,6,6],[0,2,4,4,4,5,6,6],[0,2,3,4,6,6,6,6], [0,2,3,4,6,6,6,6], [0,3,3,4,4,4,5,5], [0,3,3,4,4,4,5,5]]
# combinazione 5 
a = [ [0,2,3,4,5,6,7,7], 
     [0,2,3,4,5,6,7,7], 
     [0,2,3,4,5,6,7,7], 
     [0,2,3,4,5,6,7,7], 
     [0,3,3,3,4,5,6,6], 
     [0,3,3,3,4,5,6,6], 
     [0,3,4,4,4,4,4,4], 
     [0,3,3,3,5,5,5,5] ]

# a = [[0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5],
# [0,2,4,4,5,5,5,5]]
t_values = solve_milp(a)

if t_values:
    for e in t_values: print(e)
else: print("infeasible")


In [7]:
import ast
n = 11
file_combinazioni = f"./Data/TutteCombinazioni{n}Triang.txt"
file_combinazioni_no = f"./Data/TutteCombinazioni{n}Triang_NO.txt"
# legge le cobinazioni da file
def read_file_to_list(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        result = []
        i = 1
        for line in lines:
            #print(line)
            line = line.strip()
            if line:
                try:
                    # Convert the string representation of the list back to an actual list
                    list_in_line = ast.literal_eval(line)
                    result.append(list_in_line)
                except ValueError as e:
                    print(i)
                    print(f"Error processing line: {line}\nError: {e}")
                    break
            i += 1
            if (i % 10000 == 0): print(i)
        return result
    
result = read_file_to_list(file_combinazioni)
result_no = read_file_to_list(file_combinazioni_no)
diff = [el for el in result if el not in result_no]
for el in diff: print(el)

10000
20000
30000
40000
10000
20000
30000
40000
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_5_5_5_5_5_6_6', 2), ('3_3_3_4_6_6_7_7_7_7', 2), ('2_3_5_5_5_7_7_7_8_8', 4), ('2_3_4_6_7_7_7_7_7_7', 2)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_5_5_5_5_5_6_6', 2), ('2_4_4_5_5_5_7_7_7_7', 4), ('2_3_4_6_7_7_7_7_7_7', 4)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_4_5_5_6_6_6_6', 2), ('3_3_3_5_5_6_6_6_7_7', 2), ('2_3_5_5_6_6_7_7_7_7', 2), ('2_3_4_6_6_7_7_7_8_8', 4)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_4_5_5_6_6_6_6', 2), ('3_3_3_4_6_6_7_7_7_7', 2), ('2_3_5_5_6_6_7_7_7_7', 2), ('2_3_5_5_6_6_6_7_8_8', 2), ('2_3_4_6_6_7_7_7_8_8', 2)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_4_5_5_6_6_6_6', 2), ('2_4_4_5_5_6_6_6_7_7', 2), ('2_4_4_4_6_6_7_7_7_7', 2), ('2_3_5_5_6_6_7_7_7_7', 2), ('2_3_4_6_7_7_7_7_7_7', 2)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_4_5_5_6_6_6_6', 2), ('2_4_4_4_6_6_7_7_7_7', 4), ('2_3_5_5_6_6_7_7_7_7', 4)]
[('4_4_4_4_4_4_5_5_5_5', 1), ('3_3_4_4_5_5_5_6_7_7', 2), ('2_4_5_5_5_5_6_6_6_6', 2), ('2_4_4_5_5_5_7_7_7_7